In [33]:
import json
import pandas as pd

basepath = '/home/xilini/par-gen/01-par-gen/model/scores/'

with open('/home/xilini/par-gen/01-par-gen/data/dataset_paragraphs_v1.json', 'r') as s:
    data = json.load(s)['images']
with open(basepath + 'VISUAL-beam_2_cidertest.json', 'r') as f1:
    VISUAL = json.load(f1)
with open(basepath + 'LANGUAGE-beam_2_cidertest.json', 'r') as f2:
    LANGUAGE = json.load(f2)
with open(basepath + 'VISUAL_LANGUAGE-beam_2_cidertest.json', 'r') as f3:
    VIS_LANG = json.load(f3)

In [35]:
# choose randomly 50 images and generate json file with paths to these images

stanford_path = 'https://cs.stanford.edu/people/rak248/'
image_loc = {}
for item in data:
    if item['split'] == 'test':
        filepath = item['filepath']
        image_id = item['stanford_par_id']
        image_path = stanford_path + str(filepath) + '/' + str(image_id) + '.jpg'
        image_loc[image_id] = image_path

import random
images = list(image_loc.keys())
random.seed(4)
random.shuffle(images)

images = images[:50]

random_dict = {}
for k in images:
    if k in image_loc.keys():
        random_dict[k] = image_loc[k]

with open('/home/xilini/par-gen/01-par-gen/data/stanford_loc.json', 'w') as st:
    json.dump(random_dict, st)

In [37]:
def make_df(data_input, par_dict):
    csv_dict = {}
    csv_dict['Image'] = []
    csv_dict['Paragraph'] = []

    for item in data_input:
        if item['image_id'] in par_dict.keys():
            image_link = par_dict[item['image_id']]
            hyps = item['hypotheses']
            csv_dict['Image'].append(image_link)
            csv_dict['Paragraph'].append(hyps)
            
    return csv_dict

In [42]:
vis_df = make_df(VISUAL, random_dict)
vis_csv_df = pd.DataFrame(data=vis_df)

In [44]:
lang_df = make_df(LANGUAGE, random_dict)
lang_csv_df = pd.DataFrame(data=lang_df)

In [45]:
vis_lang_df = make_df(VIS_LANG, random_dict)
vis_lang_csv_df = pd.DataFrame(data=vis_lang_df)

In [61]:
vis_csv_df['Model'] = 'VISUAL'
lang_csv_df['Model'] = 'LANGUAGE'
vis_lang_csv_df['Model'] = 'VIS_LANG'

In [62]:
vis_csv_df.shape

(50, 3)

In [63]:
lang_csv_df.shape

(50, 3)

In [64]:
vis_lang_csv_df.shape

(50, 3)

In [65]:
final_concat = pd.concat([vis_csv_df, lang_csv_df, vis_lang_csv_df])

In [66]:
final_concat.shape

(150, 3)

In [67]:
final_concat.to_csv('./df_max_pool.csv', index=False)

In [68]:
final_concat.shape

(150, 3)

In [74]:
final_concat

,Image,Paragraph,Model
0,https://cs.stanford.edu/people/rak248/VG_100K/...,a plane is on a runway. the sky is very cloudy...,VISUAL
1,https://cs.stanford.edu/people/rak248/VG_100K/...,a man is standing on a beach. he is wearing a ...,VISUAL
2,https://cs.stanford.edu/people/rak248/VG_100K_...,a man is skiing on a hill. the man is wearing ...,VISUAL
3,https://cs.stanford.edu/people/rak248/VG_100K/...,a plane is on a runway. the airplane is blue a...,VISUAL
4,https://cs.stanford.edu/people/rak248/VG_100K/...,a woman is standing on a tennis court. she is ...,VISUAL
...,...,...,...
45,https://cs.stanford.edu/people/rak248/VG_100K_...,a man is standing on the ground. the man is we...,VIS_LANG
46,https://cs.stanford.edu/people/rak248/VG_100K_...,there are two zebras standing on the ground. t...,VIS_LANG
47,https://cs.stanford.edu/people/rak248/VG_100K_...,there is a giraffe standing in a field. there ...,VIS_LANG
48,https://cs.stanford.edu/people/rak248/VG_100K/...,a woman is walking on the sidewalk. the woman ...,VIS_LANG


In [131]:
# results for VIS, LANG, VIS + LANG
import pandas as pd
res_df = pd.read_csv('results_TEST/Batch_4155507_batch_results.csv')

In [132]:
res_df.columns

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.Image',
       'Input.Paragraph', 'Input.Model', 'Answer.objSalience_LANGUAGE',
       'Answer.objSalience_VISUAL', 'Answer.objSalience_VIS_LANG',
       'Answer.parCoherence_LANGUAGE', 'Answer.parCoherence_VISUAL',
       'Answer.parCoherence_VIS_LANG', 'Answer.sentStructure_LANGUAGE',
       'Answer.sentStructure_VISUAL', 'Answer.sentStructure_VIS_LANG',
       'Answer.wordChoice_LANGUAGE', 'Answer.wordChoice_VISUAL',
       'Answer.wordChoi

In [133]:
res_df = res_df[['Input.Image',
       'Input.Paragraph', 'Input.Model', 'Answer.objSalience_LANGUAGE',
       'Answer.objSalience_VISUAL', 'Answer.objSalience_VIS_LANG',
       'Answer.parCoherence_LANGUAGE', 'Answer.parCoherence_VISUAL',
       'Answer.parCoherence_VIS_LANG', 'Answer.sentStructure_LANGUAGE',
       'Answer.sentStructure_VISUAL', 'Answer.sentStructure_VIS_LANG',
       'Answer.wordChoice_LANGUAGE', 'Answer.wordChoice_VISUAL',
       'Answer.wordChoice_VIS_LANG']]

In [134]:
res_df_vis = res_df[['Input.Image',
       'Input.Paragraph', 'Input.Model',
       'Answer.objSalience_VISUAL', 'Answer.parCoherence_VISUAL',
       'Answer.sentStructure_VISUAL','Answer.wordChoice_VISUAL']]
res_df_vis = res_df_vis[res_df_vis['Input.Model'] == 'VISUAL']

res_df_lang = res_df[['Input.Image',
       'Input.Paragraph', 'Input.Model',
       'Answer.objSalience_LANGUAGE', 'Answer.parCoherence_LANGUAGE',
       'Answer.sentStructure_LANGUAGE','Answer.wordChoice_LANGUAGE']]
res_df_lang = res_df_lang[res_df_lang['Input.Model'] == 'LANGUAGE']

res_df_vis_lang = res_df[['Input.Image',
       'Input.Paragraph', 'Input.Model',
       'Answer.objSalience_VIS_LANG', 'Answer.parCoherence_VIS_LANG',
       'Answer.sentStructure_VIS_LANG','Answer.wordChoice_VIS_LANG']]
res_df_vis_lang = res_df_vis_lang[res_df_vis_lang['Input.Model'] == 'VIS_LANG']

In [135]:
res_df_vis.shape

(50, 7)

In [136]:
res_df_lang.shape

(50, 7)

In [137]:
res_df_vis_lang.shape

(50, 7)

In [138]:
print(res_df_vis['Answer.wordChoice_VISUAL'].mean())
print(res_df_vis['Answer.objSalience_VISUAL'].mean())
print(res_df_vis['Answer.sentStructure_VISUAL'].mean())
print(res_df_vis['Answer.parCoherence_VISUAL'].mean())

38.12
46.1
42.84
39.06


In [139]:
print(res_df_lang['Answer.wordChoice_LANGUAGE'].mean())
print(res_df_lang['Answer.objSalience_LANGUAGE'].mean())
print(res_df_lang['Answer.sentStructure_LANGUAGE'].mean())
print(res_df_lang['Answer.parCoherence_LANGUAGE'].mean())

34.52
41.38
48.38
40.5


In [140]:
print(res_df_vis_lang['Answer.wordChoice_VIS_LANG'].mean())
print(res_df_vis_lang['Answer.objSalience_VIS_LANG'].mean())
print(res_df_vis_lang['Answer.sentStructure_VIS_LANG'].mean())
print(res_df_vis_lang['Answer.parCoherence_VIS_LANG'].mean())

42.22
48.32
52.88
49.04


In [143]:
# count workers
worker_df = pd.read_csv('results_TEST/Batch_4155507_batch_results.csv')

In [144]:
worker_df.groupby('WorkerId').count()

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,Answer.parCoherence_VISUAL,Answer.parCoherence_VIS_LANG,Answer.sentStructure_LANGUAGE,Answer.sentStructure_VISUAL,Answer.sentStructure_VIS_LANG,Answer.wordChoice_LANGUAGE,Answer.wordChoice_VISUAL,Answer.wordChoice_VIS_LANG,Approve,Reject
WorkerId,,,,,,,,,,,,,,,,,,,,,
A1EUBMQ86K32XE,1,1,1,1,1,1,1,1,1,1,...,0,0,1,0,0,1,0,0,0,0
A22VLXXVIEJSD1,1,1,1,1,1,1,1,1,1,1,...,0,0,1,0,0,1,0,0,0,0
A27SMEOPKV84VI,3,3,3,3,3,3,3,3,3,3,...,1,1,1,1,1,1,1,1,0,0
A28L1K6D8QUCML,12,12,12,12,12,12,12,12,12,12,...,5,6,1,5,6,1,5,6,0,0
A3BI0AX5T5GVO3,129,129,129,129,129,129,129,129,129,129,...,43,41,45,43,41,45,43,41,0,0
A3MS21UJPMHLK0,1,1,1,1,1,1,1,1,1,1,...,0,1,0,0,1,0,0,1,0,0
A3OLRWACCCCUTU,1,1,1,1,1,1,1,1,1,1,...,0,0,1,0,0,1,0,0,0,0
AKSJ3C5O3V9RB,1,1,1,1,1,1,1,1,1,1,...,0,1,0,0,1,0,0,1,0,0
AMA18W8F60Y2J,1,1,1,1,1,1,1,1,1,1,...,1,0,0,1,0,0,1,0,0,0


In [145]:
worker_df.columns

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.Image',
       'Input.Paragraph', 'Input.Model', 'Answer.objSalience_LANGUAGE',
       'Answer.objSalience_VISUAL', 'Answer.objSalience_VIS_LANG',
       'Answer.parCoherence_LANGUAGE', 'Answer.parCoherence_VISUAL',
       'Answer.parCoherence_VIS_LANG', 'Answer.sentStructure_LANGUAGE',
       'Answer.sentStructure_VISUAL', 'Answer.sentStructure_VIS_LANG',
       'Answer.wordChoice_LANGUAGE', 'Answer.wordChoice_VISUAL',
       'Answer.wordChoi

In [146]:
worker_df['WorkTimeInSeconds'].mean()
# 7 minutes per task (?), quite a lot...

424.02666666666664

In [147]:
timings = worker_df[['AcceptTime', 'SubmitTime']]

In [148]:
timings.head()

,AcceptTime,SubmitTime
0,Tue Aug 18 05:12:02 PDT 2020,Tue Aug 18 05:12:47 PDT 2020
1,Tue Aug 18 06:46:08 PDT 2020,Tue Aug 18 06:51:26 PDT 2020
2,Tue Aug 18 06:09:32 PDT 2020,Tue Aug 18 06:12:55 PDT 2020
3,Tue Aug 18 06:25:50 PDT 2020,Tue Aug 18 06:44:39 PDT 2020
4,Tue Aug 18 06:45:20 PDT 2020,Tue Aug 18 06:50:40 PDT 2020


In [149]:
timings.loc[:, 'AcceptTime'] = pd.to_datetime(timings['AcceptTime'])
timings.loc[:, 'SubmitTime'] =  pd.to_datetime(timings['SubmitTime'])

/home/xilini/anaconda3/envs/vis-par/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/home/xilini/anaconda3/envs/vis-par/lib/python3.7/site-packages/pandas/core/indexing.py:1047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/home/xilini/anaconda3/envs/vis-par/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [150]:
timings

,AcceptTime,SubmitTime
0,2020-08-18 05:12:02,2020-08-18 05:12:47
1,2020-08-18 06:46:08,2020-08-18 06:51:26
2,2020-08-18 06:09:32,2020-08-18 06:12:55
3,2020-08-18 06:25:50,2020-08-18 06:44:39
4,2020-08-18 06:45:20,2020-08-18 06:50:40
...,...,...
145,2020-08-18 06:09:37,2020-08-18 06:14:02
146,2020-08-18 06:28:03,2020-08-18 06:45:57
147,2020-08-18 05:15:50,2020-08-18 05:16:29
148,2020-08-18 06:48:01,2020-08-18 06:55:59


In [151]:
timings['diff'] = timings['SubmitTime'] - timings['AcceptTime']

/home/xilini/anaconda3/envs/vis-par/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [152]:
diff = timings['diff']

In [153]:
diff.dt.total_seconds().min()

20.0

In [154]:
diff.dt.total_seconds().max()

1163.0

In [155]:
# run qualification thingy every 20 seconds (!)